## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
data = list(range(1,1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [3]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [5]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [6]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [8]:
# count
rdd.count()

1000

In [9]:
# first
rdd.first()

1

In [11]:
# take
rdd.take(num=20)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [12]:
# top
rdd.top(num=5)

[1000, 999, 998, 997, 996]

In [13]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [14]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([2.34771578e-01, 2.91435800e-01, 3.79045462e-01, 1.30756098e+00,
       2.52698970e+00, 2.87586264e+00, 3.78170817e+00, 2.91668538e+00,
       2.28130993e+00, 3.73090984e+00, 7.76841544e+00, 1.09390053e+01,
       3.85002839e+00, 2.38485219e+00, 7.41166091e-01, 6.09664657e+00,
       4.84672899e+00, 1.35213882e+01, 3.61432090e+00, 4.86108922e+00,
       5.98300671e+00, 1.96814124e+01, 5.15451599e+00, 2.36345831e+01,
       2.74771300e+00, 5.20767487e+00, 9.81919042e+00, 2.22138561e+01,
       1.12315037e+01, 1.49278188e+01, 1.64618309e+01, 8.33528633e+00,
       2.60313520e+01, 2.52521226e+01, 6.43919307e+00, 1.46179873e+01,
       1.82985491e+01, 1.62882746e+01, 1.33779377e+01, 4.16116513e+00,
       2.16388891e+01, 2.19284836e+01, 1.40683315e+01, 1.30666546e+01,
       4.05079382e+00, 2.46364240e+00, 3.31238210e+01, 3.56782821e+01,
       4.63233641e+01, 3.02379296e+01, 7.54589759e+00, 3.13630582e+01,
       4.20944800e+01, 6.37205266e+00, 4.35754732e+00, 4.69425487e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [15]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.collect()

[0.23477157761404688,
 0.29143580030196303,
 0.379045461529274,
 1.3075609777626775,
 2.526989699783755,
 2.875862638310104,
 3.7817081697902384,
 2.916685378593992,
 2.281309926281824,
 3.7309098404045393,
 7.768415437614492,
 10.93900527113832,
 3.8500283861983498,
 2.3848521860922665,
 0.7411660907574374,
 6.09664657408192,
 4.846728985424269,
 13.521388238007882,
 3.61432089778027,
 4.861089216622576,
 5.983006708301707,
 19.681412440791515,
 5.154515989215291,
 23.63458306455309,
 2.74771300309197,
 5.207674871255822,
 9.819190419180597,
 22.21385606441995,
 11.231503703190539,
 14.92781881090143,
 16.461830918406207,
 8.335286330207847,
 26.031352037776273,
 25.252122635179905,
 6.439193068217252,
 14.617987349922137,
 18.29854912825622,
 16.28827459581944,
 13.377937657355139,
 4.161165126472346,
 21.6388890646018,
 21.928483597789526,
 14.068331510534067,
 13.06665457317483,
 4.050793817375141,
 2.4636424037723246,
 33.1238209728046,
 35.678282075790136,
 46.32336409255451,
 30

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [16]:
def sales_tax(num):
    return num * 0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [17]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.21598985140492313,
 0.268120936277806,
 0.3487218246069321,
 1.2029560995416635,
 2.3248305238010545,
 2.645793627245296,
 3.4791715162070194,
 2.6833505483064726,
 2.098805132179278,
 3.432437053172176]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [18]:
discounted = revenue_minus_tax.map(lambda x : x*0.9)

In [19]:
discounted.take(10)

[0.19439086626443083,
 0.2413088426500254,
 0.3138496421462389,
 1.0826604895874972,
 2.092347471420949,
 2.3812142645207666,
 3.1312543645863173,
 2.4150154934758254,
 1.8889246189613504,
 3.0891933478549585]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [20]:
price_items.map(sales_tax).map(lambda x : x*0.9).top(15)

[815.0117559897423,
 810.8365164926126,
 807.4024638589419,
 791.8072323113332,
 781.9159556888928,
 755.3718687176382,
 737.978417610048,
 717.0555862613231,
 689.8030682341617,
 680.9309029059478,
 677.7713137799083,
 675.8643631031581,
 674.0408256776135,
 672.3338370532075,
 669.8981746685787]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [21]:
discounted.toDebugString()

b'(10) PythonRDD[10] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[6] at readRDDFromFile at PythonRDD.scala:274 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [23]:
mapped = price_items.map(lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.23477157761404688, 0.19439086626443083), (0.29143580030196303, 0.2413088426500254), (0.379045461529274, 0.3138496421462389), (1.3075609777626775, 1.0826604895874972), (2.526989699783755, 2.092347471420949), (2.875862638310104, 2.3812142645207666), (3.7817081697902384, 3.1312543645863173), (2.916685378593992, 2.4150154934758254), (2.281309926281824, 1.8889246189613504), (3.7309098404045393, 3.0891933478549585)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [24]:
flat_mapped = price_items.flatMap(lambda x : (x, x*0.92*0.9 ))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.23477157761404688, 0.19439086626443083, 0.29143580030196303, 0.2413088426500254, 0.379045461529274, 0.3138496421462389, 1.3075609777626775, 1.0826604895874972, 2.526989699783755, 2.092347471420949]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [25]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)


# calculate total remaining in inventory 
selected_items.count()

267

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [26]:
selected_items.reduce(lambda x,y: x + y)

122920.16248307317

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [27]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(47, 301.25266438653716),
 (30, 316.11662050630326),
 (7, 333.71864068189456),
 (17, 302.3302672600886),
 (11, 367.3551133942234),
 (12, 352.14403928382137),
 (20, 380.0860635211276)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [28]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x + y)
total_spent.take(10)

[(30, 3445.116203658355),
 (40, 1661.9220865087816),
 (50, 2258.3132680921976),
 (10, 2383.964280862706),
 (20, 1594.4536551223878),
 (31, 2461.1116118092955),
 (21, 2024.994837989994),
 (41, 1186.0634122767694),
 (1, 2691.9230499828855),
 (11, 2129.1872661560383)]

In [29]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1],ascending = False).collect()

[(45, 4626.596340714989),
 (27, 4462.4263300175735),
 (39, 4322.244773221372),
 (33, 3856.2229115247223),
 (19, 3634.97534901112),
 (30, 3445.116203658355),
 (43, 3235.494883348968),
 (38, 3228.5698267540956),
 (46, 3217.9076389573465),
 (6, 3159.7089120318574),
 (5, 3139.4021098885587),
 (42, 3006.382789502327),
 (2, 2945.130007667615),
 (3, 2774.471020552224),
 (47, 2696.4103230349338),
 (1, 2691.9230499828855),
 (44, 2673.1133499857015),
 (49, 2638.168637000291),
 (28, 2626.1081182818707),
 (34, 2541.554851357129),
 (31, 2461.1116118092955),
 (10, 2383.964280862706),
 (8, 2381.672580335725),
 (13, 2366.966552236955),
 (26, 2334.4814917198296),
 (22, 2312.4676442927034),
 (23, 2312.1260547379634),
 (24, 2277.1320826793767),
 (18, 2270.8788253638822),
 (50, 2258.3132680921976),
 (4, 2235.6256535804864),
 (11, 2129.1872661560383),
 (37, 2037.10332723915),
 (21, 2024.994837989994),
 (29, 2001.6445555190298),
 (48, 2000.811481424535),
 (14, 1971.642035760015),
 (35, 1913.902542973006),
 

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [30]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(40, 11),
 (46, 10),
 (13, 10),
 (4, 10),
 (14, 10),
 (15, 10),
 (39, 9),
 (17, 8),
 (26, 8),
 (34, 7),
 (2, 7),
 (19, 7),
 (1, 7),
 (42, 7),
 (10, 7),
 (23, 7),
 (28, 6),
 (37, 6),
 (8, 6),
 (35, 6),
 (43, 6),
 (25, 5),
 (32, 5),
 (36, 5),
 (30, 5),
 (16, 5),
 (11, 4),
 (3, 4),
 (49, 4),
 (22, 4),
 (18, 4),
 (45, 4),
 (41, 4),
 (21, 4),
 (29, 4),
 (5, 4),
 (33, 4),
 (12, 4),
 (24, 3),
 (47, 3),
 (9, 3),
 (6, 3),
 (20, 3),
 (7, 3),
 (31, 2),
 (38, 2),
 (50, 2),
 (27, 2),
 (44, 2),
 (48, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
